<a href="https://colab.research.google.com/github/HYUNMINI12/20242R0136COSE47402/blob/master/CLIP_LEARN_Monument.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# dependency 설치

!pip install pillow opencv-python albumentations rembg
!pip install onnxruntime
!pip install torch torchvision clip-by-openai
!pip install git+https://github.com/openai/CLIP.git
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 658.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 292.8 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
ERROR: Cannot install clip-by-openai==0.1.0, clip-by-openai==0.1.1, clip-by-openai==0.1.1.2, clip-by-openai==0.1.1.3, clip-by-openai=

In [ ]:
import os
import shutil
import numpy as np
from PIL import Image
import cv2
import albumentations as A
from rembg import remove
import torch
import clip
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
# 기본 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
image_size = (224, 224)  # 모델 입력 크기
base_path = '/datalab'

In [ ]:
# 기념물 이름 및 레이블 설정
monuments = {
    'invalides': "Napoleon's tomb",
    'arc_de_triomphe': "Arc de Triomphe",
    'brandenburg_gate': "Brandenburg Gate",
    'bunker_hill': "Bunker Hill Monument",
    'independence_hall': "Independence Hall",
    'liberty_bell': "Liberty Bell",
    'place_de_la_bastille': "Place de la Bastille",
    'statue_of_liberty': "Statue of Liberty"
}

In [ ]:
# 폴더 생성
for monument_name in monuments.keys():
    input_folder = os.path.join(base_path, monument_name)
    os.makedirs(input_folder, exist_ok=True)
    for suffix in ['_output', '_bg_removed', '_augmented', '_image_folder']:
        os.makedirs(os.path.join(base_path, f"{monument_name}{suffix}"), exist_ok=True)

print("모든 폴더가 생성되었습니다. 이미지를 각 monument 폴더에 추가한 후, 처리 및 학습을 진행하세요.")


모든 폴더가 생성되었습니다. 이미지를 각 monument 폴더에 추가한 후, 처리 및 학습을 진행하세요.


In [ ]:
# test 폴더 경로
test_folder_path = os.path.join(base_path, 'test')

# datalab 디렉토리와 test 폴더 생성
os.makedirs(test_folder_path, exist_ok=True)

In [ ]:
# 이미지 증강 설정
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.Blur(blur_limit=3, p=0.3)
])

In [ ]:
# 이미지 전처리 설정
def preprocess_image(image_path, processed_output_path, augmented_output_path, count=3):
    with Image.open(image_path) as img:
        # 크기 조정
        img_resized = img.resize(image_size)
        # 정규화
        img_array = np.array(img_resized) / 255.0
        img_processed = Image.fromarray((img_array * 255).astype(np.uint8))
        img_processed.save(processed_output_path)

        # OpenCV 변환 및 증강
        image_cv = cv2.cvtColor(np.array(img_resized), cv2.COLOR_RGB2BGR)
        for i in range(count):
            augmented_image = augmentations(image=image_cv)['image']
            augmented_image = Image.fromarray(cv2.cvtColor(augmented_image, cv2.COLOR_BGR2RGB))
            augmented_image.save(os.path.join(augmented_output_path, f"{os.path.splitext(os.path.basename(image_path))[0]}_aug_{i+1}.jpg"))


In [ ]:
# 배경 제거 함수
def remove_background(image_path, output_path):
    with Image.open(image_path) as img:
        img_no_bg = remove(img)
        if img_no_bg.mode == 'RGBA':
            img_no_bg = img_no_bg.convert('RGB')
        img_no_bg.save(output_path)


In [ ]:
# Dataset 클래스 정의
class ImageTextDataset(Dataset):
    def __init__(self, image_folder, text_descriptions, preprocess):
        self.image_folder = image_folder
        self.text_descriptions = text_descriptions
        self.preprocess = preprocess
        self.image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        image = self.preprocess(image)
        text = self.text_descriptions[idx]
        return image, text


In [ ]:
# 전처리 함수
def preprocess_monument_data(monument_name):
    print(f"--- {monument_name} 데이터 전처리 시작 ---")

    input_folder = os.path.join(base_path, monument_name)
    output_folder = os.path.join(base_path, f'{monument_name}_output')
    bg_removed_folder = os.path.join(base_path, f'{monument_name}_bg_removed')
    augmented_folder = os.path.join(base_path, f'{monument_name}_augmented')
    image_folder = os.path.join(base_path, f'{monument_name}_image_folder')

    image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

    for img_file in image_files:
        input_path = os.path.join(input_folder, img_file)

        # 정규화 및 증강
        processed_output_path = os.path.join(output_folder, img_file)
        preprocess_image(input_path, processed_output_path, augmented_folder)

        # 배경 제거
        bg_removed_output_path = os.path.join(bg_removed_folder, img_file)
        remove_background(input_path, bg_removed_output_path)

    for folder_name in ['output', 'bg_removed', 'augmented']:
        source_folder = os.path.join(base_path, f'{monument_name}_{folder_name}')
        for file_name in os.listdir(source_folder):
            source_file = os.path.join(source_folder, file_name)
            if os.path.isfile(source_file):
                new_file_name = f"{folder_name}_{file_name}"
                target_file = os.path.join(image_folder, new_file_name)
                shutil.move(source_file, target_file)
                print(f"파일 이동 완료: {source_file} -> {target_file}")

    print(f"--- {monument_name} 데이터 전처리 완료 ---")

In [ ]:
def train_monument_model(monument_name, text_label, preprocess, model=None, epochs=10):
    print(f"--- {monument_name} 모델 학습 시작 ---")

    # 폴더 경로 설정
    image_folder = os.path.join(base_path, f'{monument_name}_image_folder')

    # Dataset 및 DataLoader 준비
    dataset = ImageTextDataset(image_folder, [text_label] * len(os.listdir(image_folder)), preprocess)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

    # 모델 준비
    if model is None:
        model, _ = clip.load("ViT-B/32", device=device, jit=False)
        model = model.float()  # 모든 가중치를 fp32로 변환
        for param in model.parameters():
            param.requires_grad = False
        model.visual.requires_grad_(True)

    # 텍스트 임베딩 생성
    text_inputs = clip.tokenize([text_label] * len(os.listdir(image_folder))).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # 손실 함수 및 옵티마이저 설정
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.visual.parameters(), lr=1e-6, weight_decay=1e-4)

    # 학습 루프
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, _ in dataloader:
            images = images.to(device)
            image_features = model.encode_image(images)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            similarity = image_features @ text_features.T
            similarity = (similarity + 1) / 2
            target = torch.ones_like(similarity.max(dim=1)[0]).to(device)
            loss = criterion(similarity.max(dim=1)[0], target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

    print(f"--- {monument_name} 모델 학습 완료 ---")

    return model

In [ ]:
# 평가 함수
def evaluate_model_for_monument(model, preprocess, image_folder, text_inputs):
    print("--- 모델 평가 시작 ---")
    image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')]
    image_tensors = [preprocess(Image.open(img_file).convert("RGB")).unsqueeze(0).to(device) for img_file in image_files]

    model.eval()
    with torch.no_grad():
        for img_file, img_tensor, text_input in zip(image_files, image_tensors, text_inputs):
            image_features = model.encode_image(img_tensor)
            text_features = model.encode_text(text_input.unsqueeze(0))  # 차원 조정
            similarity = torch.cosine_similarity(image_features, text_features)
            print(f"이미지 파일: {img_file}, 유사도: {similarity.item():.4f}")

    print("--- 모델 평가 완료 ---")

In [ ]:
# 순차적 학습 및 평가
device = "cuda" if torch.cuda.is_available() else "cpu"
preprocess_clip = clip.load("ViT-B/32", device=device)[1]

model = None

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 108MiB/s]


In [ ]:
# Invalides 처리; /datalab의 invalides 폴더안에 10장의 사진을 넣으세요. 참고로 invalides와 Napoleon's tomb는 같습니다.
preprocess_monument_data('invalides')
model = train_monument_model('invalides', "Napoleon's tomb", preprocess_clip, model=model, epochs=6)

# 모델 평가
image_folder = os.path.join(base_path, 'invalides_image_folder')
text_inputs = clip.tokenize(["Napoleon's tomb"] * len(os.listdir(image_folder))).to(device)
evaluate_model_for_monument(model, preprocess_clip, image_folder, text_inputs)

--- invalides 데이터 전처리 시작 ---


100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 184GB/s]


파일 이동 완료: /datalab/invalides_output/Invalides_3.jpg -> /datalab/invalides_image_folder/output_Invalides_3.jpg
파일 이동 완료: /datalab/invalides_output/Invalides_9.jpg -> /datalab/invalides_image_folder/output_Invalides_9.jpg
파일 이동 완료: /datalab/invalides_output/Invalides_2.jpg -> /datalab/invalides_image_folder/output_Invalides_2.jpg
파일 이동 완료: /datalab/invalides_output/Invalides_7.jpg -> /datalab/invalides_image_folder/output_Invalides_7.jpg
파일 이동 완료: /datalab/invalides_output/Invalides_1.jpg -> /datalab/invalides_image_folder/output_Invalides_1.jpg
파일 이동 완료: /datalab/invalides_output/Invalides_5.jpg -> /datalab/invalides_image_folder/output_Invalides_5.jpg
파일 이동 완료: /datalab/invalides_output/Invalides_4.jpg -> /datalab/invalides_image_folder/output_Invalides_4.jpg
파일 이동 완료: /datalab/invalides_output/Invalides_6.jpg -> /datalab/invalides_image_folder/output_Invalides_6.jpg
파일 이동 완료: /datalab/invalides_output/Invalides_8.jpg -> /datalab/invalides_image_folder/output_Invalides_8.jpg
파일 이동 완료: 

In [ ]:
# Arc de Triomphe 처리; /datalab의 arc_de_triomphe 폴더안에 10장의 사진을 넣으세요.
preprocess_monument_data('arc_de_triomphe')
model = train_monument_model('arc_de_triomphe', "Arc de Triomphe", preprocess_clip, model=model, epochs=4)

# 모델 평가
image_folder = os.path.join(base_path, 'arc_de_triomphe_image_folder')
text_inputs = clip.tokenize(["Arc de Triomphe"] * len(os.listdir(image_folder))).to(device)
evaluate_model_for_monument(model, preprocess_clip, image_folder, text_inputs)

--- arc_de_triomphe 데이터 전처리 시작 ---
파일 이동 완료: /datalab/arc_de_triomphe_output/Arc_de_Triomphe6.jpg -> /datalab/arc_de_triomphe_image_folder/output_Arc_de_Triomphe6.jpg
파일 이동 완료: /datalab/arc_de_triomphe_output/Arc_de_Triomphe4.jpg -> /datalab/arc_de_triomphe_image_folder/output_Arc_de_Triomphe4.jpg
파일 이동 완료: /datalab/arc_de_triomphe_output/Arc_de_Triomphe5.jpg -> /datalab/arc_de_triomphe_image_folder/output_Arc_de_Triomphe5.jpg
파일 이동 완료: /datalab/arc_de_triomphe_output/Arc_de_Triomphe9.jpg -> /datalab/arc_de_triomphe_image_folder/output_Arc_de_Triomphe9.jpg
파일 이동 완료: /datalab/arc_de_triomphe_output/Arc_de_Triomphe10.jpg -> /datalab/arc_de_triomphe_image_folder/output_Arc_de_Triomphe10.jpg
파일 이동 완료: /datalab/arc_de_triomphe_output/Arc_de_Triomphe3.jpg -> /datalab/arc_de_triomphe_image_folder/output_Arc_de_Triomphe3.jpg
파일 이동 완료: /datalab/arc_de_triomphe_output/Arc_de_Triomphe8.jpg -> /datalab/arc_de_triomphe_image_folder/output_Arc_de_Triomphe8.jpg
파일 이동 완료: /datalab/arc_de_triomphe_outp

In [ ]:
# Brandenburg Gate 처리; /datalab의 bradenburg_gate 폴더안에 10장의 사진을 넣으세요.
preprocess_monument_data('brandenburg_gate')
model = train_monument_model('brandenburg_gate', "Brandenburg Gate", preprocess_clip, model=model, epochs=6)

# 모델 평가
image_folder = os.path.join(base_path, 'brandenburg_gate_image_folder')
text_inputs = clip.tokenize(["Brandenburg Gate"] * len(os.listdir(image_folder))).to(device)
evaluate_model_for_monument(model, preprocess_clip, image_folder, text_inputs)

--- brandenburg_gate 데이터 전처리 시작 ---
파일 이동 완료: /datalab/brandenburg_gate_output/Brandenburg_Gate2.jpg -> /datalab/brandenburg_gate_image_folder/output_Brandenburg_Gate2.jpg
파일 이동 완료: /datalab/brandenburg_gate_output/Brandenburg_Gate8.jpg -> /datalab/brandenburg_gate_image_folder/output_Brandenburg_Gate8.jpg
파일 이동 완료: /datalab/brandenburg_gate_output/Brandenburg_Gate6.jpg -> /datalab/brandenburg_gate_image_folder/output_Brandenburg_Gate6.jpg
파일 이동 완료: /datalab/brandenburg_gate_output/Brandenburg_Gate9.jpg -> /datalab/brandenburg_gate_image_folder/output_Brandenburg_Gate9.jpg
파일 이동 완료: /datalab/brandenburg_gate_output/Brandenburg_Gate5.jpg -> /datalab/brandenburg_gate_image_folder/output_Brandenburg_Gate5.jpg
파일 이동 완료: /datalab/brandenburg_gate_output/Bradenburg_Gate1.jpg -> /datalab/brandenburg_gate_image_folder/output_Bradenburg_Gate1.jpg
파일 이동 완료: /datalab/brandenburg_gate_output/Brandenburg_Gate10.jpg -> /datalab/brandenburg_gate_image_folder/output_Brandenburg_Gate10.jpg
파일 이동 완료: /d

In [ ]:
# Bunker Hill Monument 처리; /datalab의 bunker_hill 폴더안에 10장의 사진을 넣으세요.
preprocess_monument_data('bunker_hill')
model = train_monument_model('bunker_hill', "Bunker Hill Monument", preprocess_clip, model=model,epochs=4)

# 모델 평가
image_folder = os.path.join(base_path, 'bunker_hill_image_folder')
text_inputs = clip.tokenize(["Bunker Hill Monument"] * len(os.listdir(image_folder))).to(device)
evaluate_model_for_monument(model, preprocess_clip, image_folder, text_inputs)

--- bunker_hill 데이터 전처리 시작 ---
파일 이동 완료: /datalab/bunker_hill_output/bunker_hill2.jpg -> /datalab/bunker_hill_image_folder/output_bunker_hill2.jpg
파일 이동 완료: /datalab/bunker_hill_output/bunker_hill9.jpg -> /datalab/bunker_hill_image_folder/output_bunker_hill9.jpg
파일 이동 완료: /datalab/bunker_hill_output/bunker_hill4.jpg -> /datalab/bunker_hill_image_folder/output_bunker_hill4.jpg
파일 이동 완료: /datalab/bunker_hill_output/bunker_hill8.jpg -> /datalab/bunker_hill_image_folder/output_bunker_hill8.jpg
파일 이동 완료: /datalab/bunker_hill_output/bunker_hill6.jpg -> /datalab/bunker_hill_image_folder/output_bunker_hill6.jpg
파일 이동 완료: /datalab/bunker_hill_output/bunker_hill5.jpg -> /datalab/bunker_hill_image_folder/output_bunker_hill5.jpg
파일 이동 완료: /datalab/bunker_hill_output/bunker_hill10.jpg -> /datalab/bunker_hill_image_folder/output_bunker_hill10.jpg
파일 이동 완료: /datalab/bunker_hill_output/bunker-hill1.jpg -> /datalab/bunker_hill_image_folder/output_bunker-hill1.jpg
파일 이동 완료: /datalab/bunker_hill_output/b

In [ ]:
# Independence Hall 처리 ;/datalab의 independence_hall 폴더안에 10장의 사진을 넣으세요.
preprocess_monument_data('independence_hall')
model = train_monument_model('independence_hall', "Independence Hall", preprocess_clip, model=model, epochs=4)

# 모델 평가
image_folder = os.path.join(base_path, 'independence_hall_image_folder')
text_inputs = clip.tokenize(["Independence Hall"] * len(os.listdir(image_folder))).to(device)
evaluate_model_for_monument(model, preprocess_clip, image_folder, text_inputs)


--- independence_hall 데이터 전처리 시작 ---
파일 이동 완료: /datalab/independence_hall_output/Independence_hall10.jpg -> /datalab/independence_hall_image_folder/output_Independence_hall10.jpg
파일 이동 완료: /datalab/independence_hall_output/Independence_hall2.jpg -> /datalab/independence_hall_image_folder/output_Independence_hall2.jpg
파일 이동 완료: /datalab/independence_hall_output/Independence_hall8.jpg -> /datalab/independence_hall_image_folder/output_Independence_hall8.jpg
파일 이동 완료: /datalab/independence_hall_output/Independence_hall6.jpg -> /datalab/independence_hall_image_folder/output_Independence_hall6.jpg
파일 이동 완료: /datalab/independence_hall_output/Independence_hall3.jpg -> /datalab/independence_hall_image_folder/output_Independence_hall3.jpg
파일 이동 완료: /datalab/independence_hall_output/Independence_hall7.jpg -> /datalab/independence_hall_image_folder/output_Independence_hall7.jpg
파일 이동 완료: /datalab/independence_hall_output/Independence_hall9.jpg -> /datalab/independence_hall_image_folder/output_Inde

In [ ]:
# Liberty Bell 처리; /datalab의 liberty_bell 폴더안에 10장의 사진을 넣으세요.
preprocess_monument_data('liberty_bell')
model = train_monument_model('liberty_bell', "Liberty Bell", preprocess_clip, model=model, epochs=3)

# 모델 평가
image_folder = os.path.join(base_path, 'liberty_bell_image_folder')
text_inputs = clip.tokenize(["Liberty Bell"] * len(os.listdir(image_folder))).to(device)
evaluate_model_for_monument(model, preprocess_clip, image_folder, text_inputs)


--- liberty_bell 데이터 전처리 시작 ---
파일 이동 완료: /datalab/liberty_bell_output/Liberty_Bell6.jpg -> /datalab/liberty_bell_image_folder/output_Liberty_Bell6.jpg
파일 이동 완료: /datalab/liberty_bell_output/Liberty_Bell3.jpg -> /datalab/liberty_bell_image_folder/output_Liberty_Bell3.jpg
파일 이동 완료: /datalab/liberty_bell_output/Liberty_Bell8.jpg -> /datalab/liberty_bell_image_folder/output_Liberty_Bell8.jpg
파일 이동 완료: /datalab/liberty_bell_output/Liberty_Bell9.jpg -> /datalab/liberty_bell_image_folder/output_Liberty_Bell9.jpg
파일 이동 완료: /datalab/liberty_bell_output/Liberty_Bell5.jpg -> /datalab/liberty_bell_image_folder/output_Liberty_Bell5.jpg
파일 이동 완료: /datalab/liberty_bell_output/Liberty_Bell4.jpg -> /datalab/liberty_bell_image_folder/output_Liberty_Bell4.jpg
파일 이동 완료: /datalab/liberty_bell_output/Liberty_Bell7.jpg -> /datalab/liberty_bell_image_folder/output_Liberty_Bell7.jpg
파일 이동 완료: /datalab/liberty_bell_output/Liberty_Bell1.jpg -> /datalab/liberty_bell_image_folder/output_Liberty_Bell1.jpg
파일 이동 완료

In [ ]:
# Place de la Bastille 처리; /datalab의 place_de_la_bastille 폴더안에 10장의 사진을 넣으세요.
preprocess_monument_data('place_de_la_bastille')
model = train_monument_model('place_de_la_bastille', "Place de la Bastille", preprocess_clip, model=model, epochs =2)

# 모델 평가
image_folder = os.path.join(base_path, 'place_de_la_bastille_image_folder')
text_inputs = clip.tokenize(["Place de la Bastille"] * len(os.listdir(image_folder))).to(device)
evaluate_model_for_monument(model, preprocess_clip, image_folder, text_inputs)

--- place_de_la_bastille 데이터 전처리 시작 ---
파일 이동 완료: /datalab/place_de_la_bastille_output/Place_de_la_Bastille7.jpg -> /datalab/place_de_la_bastille_image_folder/output_Place_de_la_Bastille7.jpg
파일 이동 완료: /datalab/place_de_la_bastille_output/Place_de_la_Bastille8.jpg -> /datalab/place_de_la_bastille_image_folder/output_Place_de_la_Bastille8.jpg
파일 이동 완료: /datalab/place_de_la_bastille_output/Place_de_la_Bastille3.jpg -> /datalab/place_de_la_bastille_image_folder/output_Place_de_la_Bastille3.jpg
파일 이동 완료: /datalab/place_de_la_bastille_output/Place_de_la_Bastille2.jpg -> /datalab/place_de_la_bastille_image_folder/output_Place_de_la_Bastille2.jpg
파일 이동 완료: /datalab/place_de_la_bastille_output/Place_de_la_Bastille5.jpg -> /datalab/place_de_la_bastille_image_folder/output_Place_de_la_Bastille5.jpg
파일 이동 완료: /datalab/place_de_la_bastille_output/place_de_la_Bastille1.jpg -> /datalab/place_de_la_bastille_image_folder/output_place_de_la_Bastille1.jpg
파일 이동 완료: /datalab/place_de_la_bastille_output/P

In [ ]:
# Statue of Liberty 처리 ; /datalab의 statue_of_liberty 폴더안에 10장의 사진을 넣으세요.
preprocess_monument_data('statue_of_liberty')
model = train_monument_model('statue_of_liberty', "Statue of Liberty", preprocess_clip, model=model, epochs=2)

# 모델 평가
image_folder = os.path.join(base_path, 'statue_of_liberty_image_folder')
text_inputs = clip.tokenize(["Statue of Liberty"] * len(os.listdir(image_folder))).to(device)
evaluate_model_for_monument(model, preprocess_clip, image_folder, text_inputs)

--- statue_of_liberty 데이터 전처리 시작 ---
파일 이동 완료: /datalab/statue_of_liberty_output/Statue_of_liberty2.jpg -> /datalab/statue_of_liberty_image_folder/output_Statue_of_liberty2.jpg
파일 이동 완료: /datalab/statue_of_liberty_output/Statue_of_liberty3.jpg -> /datalab/statue_of_liberty_image_folder/output_Statue_of_liberty3.jpg
파일 이동 완료: /datalab/statue_of_liberty_output/Statue_of_liberty9.jpg -> /datalab/statue_of_liberty_image_folder/output_Statue_of_liberty9.jpg
파일 이동 완료: /datalab/statue_of_liberty_output/Statue_of_liberty5.jpg -> /datalab/statue_of_liberty_image_folder/output_Statue_of_liberty5.jpg
파일 이동 완료: /datalab/statue_of_liberty_output/Statue_of_liberty4.jpg -> /datalab/statue_of_liberty_image_folder/output_Statue_of_liberty4.jpg
파일 이동 완료: /datalab/statue_of_liberty_output/Statue_of_liberty8.jpg -> /datalab/statue_of_liberty_image_folder/output_Statue_of_liberty8.jpg
파일 이동 완료: /datalab/statue_of_liberty_output/Statue_of_liberty10.jpg -> /datalab/statue_of_liberty_image_folder/output_Statu

In [ ]:
# 모델 저장
model_path = "/content/model.pth"  # 저장할 경로 지정
torch.save(model.state_dict(), model_path)
print(f"모델이 {model_path}에 저장되었습니다.")

모델이 /content/model.pth에 저장되었습니다.


In [ ]:
# google drive에 model을 저장합니다. 이 model 경로는 이후 generating_text에서 쓰입니다.

from google.colab import drive
drive.mount('/content/drive')

# Google Drive에 저장할 경로 지정
model_path = "/content/drive/MyDrive/model.pth"

# 모델 저장
torch.save(model.state_dict(), model_path)
print(f"모델이 {model_path}에 저장되었습니다.")

Mounted at /content/drive
모델이 /content/drive/MyDrive/model.pth에 저장되었습니다.


In [ ]:
# Google Drive에서 모델 로드
model.load_state_dict(torch.load(model_path))
model.eval()  # 평가 모드로 전환
print(f"모델이 {model_path}에서 로드되었습니다.")

<ipython-input-26-d06529ee5af1>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


모델이 /content/drive/MyDrive/model.pth에서 로드되었습니다.


In [ ]:
# /datalab안의 test폴더에 사진을 넣은 후, 사진의 경로를 image_path에 넣어주시면 됩니다.

import torch
import clip
from PIL import Image
from torchvision import transforms
import os

# 모델과 전처리 함수 불러오기
device = "cuda" if torch.cuda.is_available() else "cpu"
model_clip, preprocess_clip = clip.load("ViT-B/32", device=device)

# 저장된 모델 로드
model_path = "/content/drive/MyDrive/model.pth"
model_clip.load_state_dict(torch.load(model_path))
model_clip.eval()  # 평가 모드로 설정

# 기념물 레이블 리스트
monuments = {
    'invalides': "Napoleon's tomb",
    'arc_de_triomphe': "Arc de Triomphe",
    'brandenburg_gate': "Brandenburg Gate",
    'bunker_hill': "Bunker Hill Monument",
    'independence_hall': "Independence Hall",
    'liberty_bell': "Liberty Bell",
    'place_de_la_bastille': "Place de la Bastille",
    'statue_of_liberty': "Statue of Liberty"
}

# 이미지 경로
image_path = "/datalab/test/Statue_of_liberty2.jpg"  # 테스트할 이미지 경로

# 이미지 전처리
image = Image.open(image_path).convert("RGB")
image_input = preprocess_clip(image).unsqueeze(0).to(device)

# 텍스트 레이블에 대한 전처리
text_labels = list(monuments.values())
text_inputs = clip.tokenize(text_labels).to(device)

# 모델을 통해 이미지와 텍스트의 특징 벡터를 얻음
with torch.no_grad():
    image_features = model_clip.encode_image(image_input)
    text_features = model_clip.encode_text(text_inputs)

    # 특징 벡터를 정규화
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # 이미지와 각 기념물 텍스트 간의 유사도 계산
    similarity = (image_features @ text_features.T).squeeze(0)

# 유사도가 가장 높은 기념물 찾기
best_match_idx = similarity.argmax().item()
best_match_label = text_labels[best_match_idx]
best_match_similarity = similarity[best_match_idx].item()

print(f"이미지: {image_path}에서 가장 유사한 기념물은 '{best_match_label}'이고, 유사도는 {best_match_similarity:.4f}입니다.")

<ipython-input-27-a9c4abaf9e50>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_clip.load_state_dict(torch.load(model_path))


이미지: /datalab/test/Statue_of_liberty2.jpg에서 가장 유사한 기념물은 'Statue of Liberty'이고, 유사도는 0.6470입니다.


In [ ]:
# datalab의 Test 폴더에 사진을 넣고 실행시키면 각 사진에 대한 유물 이름이 출력됩니다.

import torch
import clip
from PIL import Image
from torchvision import transforms
import os

# 모델과 전처리 함수 불러오기
device = "cuda" if torch.cuda.is_available() else "cpu"
model_clip, preprocess_clip = clip.load("ViT-B/32", device=device)

# 저장된 모델 로드
model_path = "/content/drive/MyDrive/model.pth"
model_clip.load_state_dict(torch.load(model_path))
model_clip.eval()  # 평가 모드로 설정

# 기념물 레이블 리스트
monuments = {
    'invalides': "Napoleon's tomb",
    'arc_de_triomphe': "Arc de Triomphe",
    'brandenburg_gate': "Brandenburg Gate",
    'bunker_hill': "Bunker Hill Monument",
    'independence_hall': "Independence Hall",
    'liberty_bell': "Liberty Bell",
    'place_de_la_bastille': "Place de la Bastille",
    'statue_of_liberty': "Statue of Liberty"
}

# 텍스트 레이블에 대한 전처리
text_labels = list(monuments.values())
text_inputs = clip.tokenize(text_labels).to(device)

# /datalab/test/ 경로에 있는 모든 JPG 파일 처리
image_dir = "/datalab/test/"
image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]

# 결과 저장
results = []

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)

    # 이미지 전처리
    image = Image.open(image_path).convert("RGB")
    image_input = preprocess_clip(image).unsqueeze(0).to(device)

    # 모델을 통해 이미지와 텍스트의 특징 벡터를 얻음
    with torch.no_grad():
        image_features = model_clip.encode_image(image_input)
        text_features = model_clip.encode_text(text_inputs)

        # 특징 벡터를 정규화
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        # 이미지와 각 기념물 텍스트 간의 유사도 계산
        similarity = (image_features @ text_features.T).squeeze(0)

    # 유사도가 가장 높은 기념물 찾기
    best_match_idx = similarity.argmax().item()
    best_match_label = text_labels[best_match_idx]
    best_match_similarity = similarity[best_match_idx].item()

    # 결과 저장
    results.append({
        "image": image_file,
        "best_match": best_match_label,
        "similarity": best_match_similarity
    })

# 결과 출력
for result in results:
    print(f"이미지 '{result['image']}'에서 가장 유사한 기념물은 '{result['best_match']}'이고, 유사도는 {result['similarity']:.4f}입니다.")


<ipython-input-32-9671b6007431>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_clip.load_state_dict(torch.load(model_path))


이미지 'Arc_de_Triomphe6.jpg'에서 가장 유사한 기념물은 'Arc de Triomphe'이고, 유사도는 0.8320입니다.
이미지 'bunker_hill2.jpg'에서 가장 유사한 기념물은 'Bunker Hill Monument'이고, 유사도는 0.8340입니다.
이미지 'Independence_hall10.jpg'에서 가장 유사한 기념물은 'Independence Hall'이고, 유사도는 0.8457입니다.
이미지 'Arc_de_Triomphe4.jpg'에서 가장 유사한 기념물은 'Arc de Triomphe'이고, 유사도는 0.8198입니다.
이미지 'bunker_hill9.jpg'에서 가장 유사한 기념물은 'Bunker Hill Monument'이고, 유사도는 0.8193입니다.
이미지 'bunker_hill4.jpg'에서 가장 유사한 기념물은 'Bunker Hill Monument'이고, 유사도는 0.8369입니다.
이미지 'bunker_hill8.jpg'에서 가장 유사한 기념물은 'Bunker Hill Monument'이고, 유사도는 0.8496입니다.
이미지 'Place_de_la_Bastille7.jpg'에서 가장 유사한 기념물은 'Place de la Bastille'이고, 유사도는 0.8677입니다.
이미지 'Independence_hall2.jpg'에서 가장 유사한 기념물은 'Independence Hall'이고, 유사도는 0.8359입니다.
이미지 'Independence_hall8.jpg'에서 가장 유사한 기념물은 'Independence Hall'이고, 유사도는 0.8472입니다.
이미지 'Brandenburg_Gate2.jpg'에서 가장 유사한 기념물은 'Brandenburg Gate'이고, 유사도는 0.8306입니다.
이미지 'bunker_hill6.jpg'에서 가장 유사한 기념물은 'Bunker Hill Monument'이고, 유사도는 0.7964입니다.
이미지 'Place_de_la_Bastille8.jpg'에서 

In [ ]:
# test 폴더에 사진 파일을 넣고 image_path에 경로를 설정해주면 됩니다.

import torch
import clip
from PIL import Image
import os

# 모델과 전처리 함수 불러오기
device = "cuda" if torch.cuda.is_available() else "cpu"
model_clip, preprocess_clip = clip.load("ViT-B/32", device=device)

# 저장된 모델 로드
model_path = "/content/drive/MyDrive/model.pth"
model_clip.load_state_dict(torch.load(model_path))
model_clip.eval()  # 평가 모드로 설정

# 기념물 레이블 리스트
monuments = {
    'invalides': "Napoleon's tomb",
    'arc_de_triomphe': "Arc de Triomphe",
    'brandenburg_gate': "Brandenburg Gate",
    'bunker_hill': "Bunker Hill Monument",
    'independence_hall': "Independence Hall",
    'liberty_bell': "Liberty Bell",
    'place_de_la_bastille': "Place de la Bastille",
    'statue_of_liberty': "Statue of Liberty"
}

# 이미지 경로
image_path = "/datalab/test/Invalides_7.jpg"  # 테스트할 이미지 경로

# 이미지 전처리
image = Image.open(image_path).convert("RGB")
image_input = preprocess_clip(image).unsqueeze(0).to(device)

# 텍스트 레이블에 대한 전처리
text_labels = list(monuments.values())
text_inputs = clip.tokenize(text_labels).to(device)

# 모델을 통해 이미지와 텍스트의 특징 벡터를 얻음
with torch.no_grad():
    image_features = model_clip.encode_image(image_input)
    text_features = model_clip.encode_text(text_inputs)

    # 특징 벡터를 정규화
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # 이미지와 각 기념물 텍스트 간의 유사도 계산
    similarity = (image_features @ text_features.T).squeeze(0)

# 모든 항목에 대한 유사도 출력
for i, label in enumerate(text_labels):
    print(f"기념물: {label} | 유사도: {similarity[i].item():.4f}")

<ipython-input-41-be4b2667e976>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_clip.load_state_dict(torch.load(model_path))


기념물: Napoleon's tomb | 유사도: 0.5840
기념물: Arc de Triomphe | 유사도: 0.5703
기념물: Brandenburg Gate | 유사도: 0.5146
기념물: Bunker Hill Monument | 유사도: 0.5635
기념물: Independence Hall | 유사도: 0.5566
기념물: Liberty Bell | 유사도: 0.6079
기념물: Place de la Bastille | 유사도: 0.5508
기념물: Statue of Liberty | 유사도: 0.5732
